In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### DataSet
For this analysis, we will be using Zomato Bangalore Restaurants dataset present on kaggle. The dataset contains all the details of the restaurants listed on Zomato website as of 15th March 2019.

#### About Zomato
Zomato is an Indian restaurant search and discovery service founded in 2008 by Deepinder Goyal and Pankaj Chaddah. It currently operates in 24 countries. It provides information and reviews of restaurants, including images of menus where the restaurant does not have its own website and also online delivery.

#### Problem statement
The basic idea of analyzing the Zomato dataset is to get a fair idea about the factors affecting the establishment of different types of restaurants at different places in Bengaluru. This Zomato data aims at analyzing demography of the location. Most importantly it will help new restaurants in deciding their theme, menus, cuisine, cost, etc for a particular location. It also aims at finding similarity between neighborhoods of Bengaluru on the basis of food.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('dark_background')

In [ ]:
df = pd.read_csv('../input/zomato-bangalore-restaurants/zomato.csv')
df.head()

In [ ]:
df.shape

##### Data Cleaning

In [ ]:
df.columns

In [ ]:
df.dtypes

##### Here , we can see that only 'votes' is only integer so we need to change 'rate' ,'votes' and 'approx_cost(for two people)' to integer.

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

#### There are missing values in 'Rate' , 'Phone' , 'location', 'rest_type', 'dish_liked', 'cuisines', 'approx_cost(for two people)'. 

We will drop some columns that are not needed here phone, dish_liked

In [ ]:
zm=df.drop(['url','address','dish_liked','phone','menu_item','reviews_list'] ,axis =1)
zm.head()

In [ ]:
zm.shape

Firstly remove the duplicates from dataset if any

In [ ]:
zm.drop_duplicates(inplace = True)
zm.shape

##### Cleaning Rate column

In [ ]:
zm['rate'].unique()

** Remove 'New', '/5' and '-' as it is  convenient

In [ ]:
def ratehandling(value):
    if(value == 'NEW' or value =='-'):
        return np.nan
    else:
        value = str(value).split('/')
        value =value[0]
        return float(value)

In [ ]:
zm['rate'] =zm['rate'].apply(ratehandling)
zm.head()

In [ ]:
zm['rate']

Replace null values with mean in rate column 

In [ ]:
zm['rate'].fillna(zm['rate'].mean(),inplace= True)
zm['rate'].isnull().sum()

In [ ]:
zm.info()

Drop null values

In [ ]:
zm.dropna(inplace = True)
zm.head()

In [ ]:
zm['location'].unique()

In [ ]:
zm['listed_in(city)'].unique()

here we can see that 'location' and 'Listed_in(city)' both are there. we need only one


In [ ]:
zm.head()

In [ ]:
zm = zm.drop(['listed_in(city)'], axis =1 )

In [ ]:
zm.head()

In [ ]:
zm['approx_cost(for two people)'].unique()

Here we can see that these values contains ',' in some values so we need to remove it 

In [ ]:
def handlecomma(value):
    value = str(value)
    if ',' in value:
        value = value.replace(',','')
        return float(value)
    else:
        return float(value)


In [ ]:
zm['approx_cost(for two people)'] = zm['approx_cost(for two people)'].apply(handlecomma) 

In [ ]:
zm['approx_cost(for two people)'].unique()

now we can see every value is converted into float 

In [ ]:
zm.head()

handling the rest type column

In [ ]:
zm['rest_type'].unique()

In [ ]:
zm['rest_type'].value_counts()

here what i can do is that those resturant type which are having 1 2  types i can put them in a cluster

In [ ]:
rest_types = zm['rest_type'].value_counts(ascending=False)
rest_types

now will make rest_types <1000 as 'others'

In [ ]:
rest_lessthan1000 = rest_types[rest_types<1000]
rest_lessthan1000 

In [ ]:
def handlerest_type(value):
    if(value in rest_lessthan1000):
        return 'others'
    else:
        return value

In [ ]:
zm['rest_type']= zm['rest_type'].apply(handlerest_type)
zm['rest_type'].value_counts()

In [ ]:
zm.head()

In [ ]:
zm['location'].unique()

In [ ]:
zm['location'].value_counts()

In [ ]:
zm['location'].value_counts()

In [ ]:
location = zm['location'].value_counts(ascending= False)

In [ ]:
location_lessthan500 = location[location<500]
location_lessthan500

In [ ]:
def handle_location(value):
    if(value in location_lessthan500):
        return 'others'
    else:
        return value

In [ ]:
zm['location'] = zm['location'].apply(handle_location)
zm['location'].value_counts()

In [ ]:
zm.head()

In [ ]:
zm['cuisines'].value_counts()

In [ ]:
cuisines = zm['cuisines'].value_counts(ascending = False)
cuisines_lessthan200 = cuisines[cuisines<200]

def handle_cuisines(value):
    if(value in cuisines_lessthan200):
        return 'others'
    else:
        return value

In [ ]:
zm['cuisines'] = zm['cuisines'].apply(handle_cuisines)
zm['cuisines'].value_counts()